# Cardiotocographic Classification using Spark
##### Cardiotocographic classification for fetal heart-rate and uterine contractions, implemented with a PySpark Pipeline

Dataset from the UCI data repository: https://archive.ics.uci.edu/ml/datasets/cardiotocography

In [42]:
#Pipeline dependencies
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import SparkSession

#Data manipulation, analysis and plotting tools
import pandas as pd
import numpy as np
import matplotlib as plt

#Machine Learning libraries
#import sklearn as sk
#import tensorflow as tf

In [18]:
#Define the session and cluster
spark = SparkSession.builder \
                    .master('local[4]') \
                    .appName('cardiotocography_pipeline') \
                    .getOrCreate()

In [28]:
#Read the CSV dataset file and print the schema
df = spark.read.options(header='True', #Specify that headers exist in dataset
                        inferSchema='True', 
                        delimiter=',' #Comma delimited
                       ).csv("CTG_data.csv") #Source file

#Drop missing values
df = df.dropna()

#Required to drop some attributes which likely do not add to the learning of the model:
#- FileName, Date, SegFile to be truncated
df = df.drop("FileName","Date","SegFile")

#Dataset has seemed to already One-Hot-Encoded the multiclass target 'Class' labels to:
# A,B,C,D,E,AD,DE,LD,FS,SUSP
#Since this is the target label, we can remove those columns and retain the Class column
#which contains the multiclass numbers between 1-10
df = df.drop("A","B29","C","D","E32","AD","DE","LD","FS","SUSP")
df.printSchema()

root
 |-- b3: integer (nullable = true)
 |-- e4: integer (nullable = true)
 |-- LBE: integer (nullable = true)
 |-- LB: integer (nullable = true)
 |-- AC: integer (nullable = true)
 |-- FM: integer (nullable = true)
 |-- UC: integer (nullable = true)
 |-- ASTV: integer (nullable = true)
 |-- MSTV: double (nullable = true)
 |-- ALTV: integer (nullable = true)
 |-- MLTV: double (nullable = true)
 |-- DL: integer (nullable = true)
 |-- DS: integer (nullable = true)
 |-- DP: integer (nullable = true)
 |-- DR: integer (nullable = true)
 |-- Width: integer (nullable = true)
 |-- Min: integer (nullable = true)
 |-- Max: integer (nullable = true)
 |-- Nmax: integer (nullable = true)
 |-- Nzeros: integer (nullable = true)
 |-- Mode: integer (nullable = true)
 |-- Mean: integer (nullable = true)
 |-- Median: integer (nullable = true)
 |-- Variance: integer (nullable = true)
 |-- Tendency: integer (nullable = true)
 |-- CLASS: integer (nullable = true)
 |-- NSP: integer (nullable = true)



### Feature descriptions:
- b: Start instant
- e: End instant
- LBE: Baseline value (medical expert)
- LB: Baseline value (SisPorto)
- AC: Accelerations (SisPorto)
- FM: Foetal movement (SisPorto)
- UC: Uterine contractions (SisPorto)
- ASTV: percentage of time with abnormal short term variability  (SisPorto)
- mSTV:	mean value of short term variability  (SisPorto)
- ALTV:	percentage of time with abnormal long term variability  (SisPorto)
- mLTV:	mean value of long term variability  (SisPorto)
- DL:	light decelerations
- DS:	severe decelerations
- DP:	prolongued decelerations
- DR:	repetitive decelerations
- Width:  histogram width
- Min:	low freq. of the histogram
- Max:	high freq. of the histogram
- Nmax:	number of histogram peaks
- Nzeros:	number of histogram zeros
- Mode:	histogram mode
- Mean:	histogram mean
- Median:	histogram median
- Variance:	histogram variance
- Tendency:	histogram tendency: [-1=left assymetric; 0=symmetric; 1=right assymetric]

### Classes
- A: Calm sleep
- B: REM sleep
- C: Calm vigilance
- D: Actice vigilance
- E: Shift pattern (A or Susp with shifts)
- AD: Accelerative/Decelerative pattern (stress simulation)
- DE: Decelerative pattern (vagal stimulation)
- LD: Largely develerative pattern
- FS: Flat-sinusoidal pattern (pathogenic state)
- SUSP: Suspect pattern

#### Further higher level classification (NSP):
- 1: Normal
- 2: Suspect
- 3: Pathogenic

In [62]:
featuresToScale = [f[0] for f in df.dtypes if f[0] not in ["CLASS", "NSP"]]

assembler = VectorAssembler(inputCols=featuresToScale, outputCol="x_vec")
temp_train = assembler.transform(df)

scaler = MinMaxScaler(inputCol="x_vec", outputCol="x_scaled")
scaledData = scaler.fit(temp_train).transform(temp_train)

In [64]:
scaledData.select("x_scaled").show(20, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|x_scaled                                                                                                                                                                                                                                                                                                                                                                                        |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------